In [1]:
# ! git clone https://github.com/nachi-hebbar/Transfer-Learning-ResNet-Keras.gitInvalidArgumentError: Graph execution error:

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.layers.experimental.preprocessing import Rescaling


from glob import glob

from keras.layers import BatchNormalization

from tensorflow.keras.layers import Dropout

from keras.callbacks import EarlyStopping
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

2023-03-10 10:17:31.172314: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 10:17:32.081194: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 10:17:32.081250: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-10 10:17:35.041747: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
print(tf.__version__)

2.11.0


In [4]:
my_dir = r'/home/ubuntu/Downloads/rajsir_data/Classification_number_data/number_plate data_classification/'

train_dir = os.path.join(my_dir, 'train')

val_dir =  os.path.join(my_dir, 'val')

## Find Useless Image
  Which Image has graph is not plotted properly

In [5]:
# import PIL
# from pathlib import Path
# from PIL import UnidentifiedImageError

# path = Path(r'/home/ubuntu/Downloads/cv/number_plate data/train').rglob("*.jpg")
# for img_p in path:
#     try:
#         img = PIL.Image.open(img_p)
#     except PIL.UnidentifiedImageError:
#             print(img_p)

In [6]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# IMAGE_SIZE = 224
# batch_size = 36

# datagen = ImageDataGenerator(rescale=1.0/255)

# train_ds = datagen.flow_from_directory(train_dir,
#                                         target_size = (IMAGE_SIZE,IMAGE_SIZE),
#                                         batch_size = batch_size,
#                                         class_mode = 'categorical')

# val_ds = datagen.flow_from_directory(val_dir,
#                                         target_size = (IMAGE_SIZE,IMAGE_SIZE),
#                                         batch_size = batch_size,
#                                         class_mode = 'categorical')

In [5]:
img_height,img_width = 224,224

batch_size = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  labels='inferred',  
  validation_split = 0.17,
  subset = "training",
  seed = 123,
  image_size = (img_height, img_width),
  shuffle = True,  
  label_mode = 'int',                         # for int use sparse_categorical_crossentropy
  batch_size = batch_size)

Found 359627 files belonging to 36 classes.
Using 298491 files for training.


2023-03-10 10:18:06.860275: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-10 10:18:06.860316: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-10 10:18:06.860346: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-OptiPlex-3040): /proc/driver/nvidia/version does not exist
2023-03-10 10:18:06.868991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  val_dir,
  labels='inferred',  
  validation_split = 0.17,
  subset = "validation",
  seed = 123,
  image_size = (img_height, img_width),
  shuffle = True,
  label_mode='int',  
  batch_size = batch_size)

Found 87988 files belonging to 36 classes.
Using 14957 files for validation.


In [9]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# train_datagen = ImageDataGenerator( rescale = 1.0/255. )
# test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

In [7]:
# class_names = train_ds.class_indices
class_names = train_ds.class_names
print(class_names)

['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9', 'class_A', 'class_B', 'class_C', 'class_D', 'class_E', 'class_F', 'class_G', 'class_H', 'class_I', 'class_J', 'class_K', 'class_L', 'class_M', 'class_N', 'class_O', 'class_P', 'class_Q', 'class_R', 'class_S', 'class_T', 'class_U', 'class_V', 'class_W', 'class_X', 'class_Y', 'class_Z']


In [8]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(16, 224, 224, 3)
(16,)


In [12]:
# FOR COLOR CHANNEL IMAGE

# normalization_layer = tf.keras.layers.Rescaling(1./255)

In [13]:
# train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

# normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))
# first_image = image_batch[0]

# # Notice the pixel values are now in `[0,1]`.
# print(np.min(first_image), np.max(first_image))

## Configuring the dataset for better performance
Buffered prefetching can be used to ensure that the data can be taken from disk without having I/O become blocking. 
Dataset.cache() keeps the images in memory after they have been loaded off disk during the first epoch. Dataset.prefetch() will overlap the data preprocessing and model execution while training.


In [9]:
# AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
folders = glob(train_dir + "/*")
K = len(folders)
K

36

In [11]:
resnet_model = Sequential()

# pretrained_model= tf.keras.applications.InceptionResNetV2(include_top = False,

pretrained_model= tf.keras.applications.ResNet50(include_top = False,
#                    input_shape = (224,224,3),        #   + [3], Making the image into 3 Channel, so concating 3.
                   pooling = 'max',
                   classes = 36,
                   weights = 'imagenet')

for layer in pretrained_model.layers:                 # pretrained_model.trainable = False
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
# resnet_model.add(Rescaling(1./255))
# resnet_model.add(Dense(512, activation='relu'))
# resnet_model.add(Dropout(0.3))
# resnet_model.add(BatchNormalization())
# resnet_model.add(Dense(256, activation='relu'))
# resnet_model.add(Dropout(0.25))
# resnet_model.add(BatchNormalization())
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(Dropout(0.2))
resnet_model.add(BatchNormalization())
resnet_model.add(Dense(K, activation='softmax'))

In [12]:
# Create the model
# model = Sequential()
# model.add(VGG16(weights="imagenet", include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
# model.add(Flatten())
# model.add(Dense(128, activation="relu"))
# model.add(Dense(128, activation="relu"))
# model.add(Dense(64, activation="relu"))
# model.add(Dense(4, activation="sigmoid"))

# model.layers[-6].trainable = False

# model.summary()

In [13]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper (ModuleWrapp  (None, 2048)             0         
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 128)              262272    
 pper)                                                           
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                        

In [20]:
# from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers.legacy import Adam

# resnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

resnet_model.compile(loss = 'sparse_categorical_crossentropy', optimizer =  Adam(learning_rate=0.01) , metrics=['accuracy'])

In [21]:
batch_size = 2

earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 2, verbose=0, mode = 'auto')

In [ ]:
import time

start = time.time()

history = resnet_model.fit(train_ds, epochs = 1, batch_size = batch_size, validation_data = val_ds, callbacks = [earlystop])

stop = time.time()

print(f'Training took: {(stop-start)/60} minutes')

  899/18656 [>.............................] - ETA: 7:08:06 - loss: 1.7349 - accuracy: 0.4503

In [ ]:
# epochs=1
# history = resnet_model.fit(
#   train_ds,
#   validation_data=val_ds,
#   epochs=epochs
# )

In [2]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.grid()
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.grid()
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [ ]:
# Test
scores = resnet_model.evaluate(val_ds, verbose=0)
print(scores)
print("Score : %.2f%%" % (scores[1]*100))

In [ ]:
test_loss, test_accuracy = resnet_model.evaluate(val_ds, steps=int(100))

print("Test results \n Loss:",test_loss,'\n Accuracy',test_accuracy)

In [ ]:
import cv2
image=cv2.imread('/home/ubuntu/Downloads/rahul/char_165_671604_1674814554496_7.jpg')
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)

In [ ]:
#  from tensorflow.keras.preprocessing import image
#     img = image.load_img(path + image_name, target_size = IMAGE_SIZE)
#     x = image.img_to_array(img)
#     x = x / 255
#     x = np.expand_dims(x, axis = 0)
#     img_data = preprocess_input(x)
#     a = np.argmax(model.predict(img_data), axis = 1)

In [ ]:
Y_pred = resnet_model.predict(image)

print(Y_pred)

In [ ]:
output_class = class_names[np.argmax(Y_pred)]
print("The predicted class is", output_class)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

In [ ]:
Y_pred1 = resnet_model.predict(val_ds)

In [ ]:
Y_pred1

In [ ]:
y_pred1 = np.argmax(Y_pred, axis=1)
y_pred1

In [ ]:
print(classification_report(val_ds.classes, y_pred1))

In [ ]:
classification_report(validation_generator.classes, y_pred, target_names=target_names)

In [ ]:
# conf_matrix = confusion_matrix(train_ds, Y_pred)
# conf_matrix

## Creating an image

In [ ]:
from PIL import Image, ImageDraw, ImageFont
 
img = Image.new('RGB', (24, 44), color = (255, 255, 255))
 
# fnt = ImageFont.truetype('/Library/Fonts/Arial.ttf', 15)  # selecting the font
d = ImageDraw.Draw(img)
d.text((11,11), "K", fill = (0, 0, 0))
 
img.save('K.png')

## Extract coordinate from xml

In [ ]:
from glob import glob
import xml.etree.ElementTree as xet


path = glob('/home/ubuntu/Downloads/CVAT/*annotations.xml')

labels_dict = dict(FileName=[], Xmin=[], Xmax=[], Ymin=[], Ymax=[], ClassName=[])

for filename in path:
    print(filename)

    info = xet.parse(filename)
    print("info ",info)
    
    root = info.getroot()
    print("root ",root)
    
    for box in root.findall('./image/box'):
        Xmin = box.attrib['xtl']
        Xmax = box.attrib['xbr']
        Ymin = box.attrib['ytl']
        Ymax = box.attrib['ybr']
        ClassName = box.attrib['label']
#         print(box.attrib)

    
        for  image in root.iter('image'): 
            FileName = image.attrib['name']
            print(image.attrib)
            print(FileName)
        labels_dict['FileName'].append(FileName)
        labels_dict['Xmin'].append(Xmin)
        labels_dict['Xmax'].append(Xmax)
        labels_dict['Ymin'].append(Ymin)
        labels_dict['Ymax'].append(Ymax)
        labels_dict['ClassName'].append(ClassName)
        
#          print("aaa",labels_dict)

In [ ]:
import pandas as pd
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df

# df.to_csv('annotation.csv')

## Image Augmentation

In [ ]:
df.to_csv('annotation.csv')